In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()
import kglab
import pandas as pd
from pandas import DataFrame

# Core Functions

This page defines all boilerplate functions to be used when analyzing sboms.  To see these functions in action please view the following pages.

Many of the queries seen on this page were written with the help of ChatGPT.  To repeat this simply prompt ChatGPT as follows: 
`Write me a SPARQL query to select X, Y, and Z`

#| hide
### Load KG for testing

In [ ]:
#| hide
import kglab
kg = kglab.KnowledgeGraph()
kg.load_rdf("sboms/rdf/model.rdf.xml", format="xml")

<kglab.kglab.KnowledgeGraph>

## Basic Queries

These functions perform basic queries and return metadata about a given knowledge graph

In [ ]:
#| export
from kglab import KnowledgeGraph
def show_metadata(kg:KnowledgeGraph, #Knowledge graph to query from
                  dataframe: bool=False): #Optionally, return result to dataframe
    """
    Return total number of triples, distict entities, and properties to a pandas dataframe.
    """
    
    query = """
    SELECT 
        (COUNT(*) AS ?triples)
        (COUNT(DISTINCT ?entity) AS ?entities)
        (COUNT(DISTINCT ?property) AS ?properties)
    WHERE {
        ?subject ?property ?object .
        BIND(?subject AS ?entity) .
    }
    """

    if dataframe:
        return kg.query_as_df(query)

    for row in kg.query(query):
        print("Total Triples:", row['triples'])
        print("Distinct Entities:", row['entities'])
        print("Distinct Properties:", row['properties'])

In [ ]:
#| hide
show_metadata(kg)

Total Triples: 306
Distinct Entities: 56
Distinct Properties: 62


In [ ]:
#| export
from kglab import KnowledgeGraph
def show_entity_types(kg: KnowledgeGraph, # Knowledge graph to query from 
                      dataframe: bool=False): # Optionally, return result to dataframe
    """
    Show counts of entity types
    """
    
    query = """
    SELECT ?type (COUNT(?entity) as ?count)
    WHERE {
        ?entity a ?type .
    }
    GROUP BY ?type
    ORDER BY DESC(?count)
    """
    
    if dataframe:
        return kg.query_as_df(query)

    result = kg.query(query)
    for row in result:
        print(row["type"], ":", row["count"])


In [ ]:
#| hide
show_entity_types(kg)

http://spdx.org/rdf/terms#Relationship : 11
http://spdx.org/rdf/terms#Checksum : 10
http://spdx.org/rdf/terms#ExtractedLicensingInfo : 5
http://spdx.org/rdf/terms#Annotation : 5
http://spdx.org/rdf/terms#Package : 4
http://spdx.org/rdf/terms#File : 4
http://spdx.org/rdf/terms#ExternalRef : 3
http://spdx.org/rdf/terms#DisjunctiveLicenseSet : 2
http://www.w3.org/2009/pointers#StartEndPointer : 2
http://www.w3.org/2009/pointers#ByteOffsetPointer : 2
http://www.w3.org/2009/pointers#LineCharPointer : 2
http://spdx.org/rdf/terms#SpdxDocument : 1
http://spdx.org/rdf/terms#PackageVerificationCode : 1
http://spdx.org/rdf/terms#ConjunctiveLicenseSet : 1
http://spdx.org/rdf/terms#CreationInfo : 1
http://spdx.org/rdf/terms#ExternalDocumentRef : 1
http://spdx.org/rdf/terms#Snippet : 1


In [ ]:
#| export
from kglab import KnowledgeGraph
def show_top_n_props(kg: KnowledgeGraph,  # Knowledge graph to query from
                     n: int=10, # Top n results to display
                     dataframe: bool=False): # Optionally, return result to dataframe
    """
    Display the top N properties from a knowledge graph
    """
    
    query = """
    SELECT ?property (COUNT(?property) as ?count)
    WHERE {
        ?s ?property ?o .
    }
    GROUP BY ?property
    ORDER BY DESC(?count)
    LIMIT %d
    """ % n
    
    if dataframe:
        return kg.query_as_df(query)

    result = kg.query(query)
    for row in result:
        print(row["property"], ":", row["count"])

In [ ]:
#| hide
show_top_n_props(kg, 10, dataframe=True)

,property,count
0,rdf:type,56
1,rdfs:comment,14
2,spdx:relatedSpdxElement,11
3,spdx:fileContributor,11
4,spdx:relationship,11
5,spdx:relationshipType,11
6,spdx:checksumValue,10
7,spdx:checksum,10
8,spdx:algorithm,10
9,spdx:licenseConcluded,9


In [ ]:
#| export
from kglab import KnowledgeGraph
def show_measures(kg:KnowledgeGraph): #Knowledge graph to query from
    """
    Display number of nodes and edges for knowledge graph
    """
    measure = kglab.Measure()
    measure.measure_graph(kg)

    print("edges", measure.get_edge_count())
    print("nodes", measure.get_node_count())

## Files

These functions peform queries on a SBOM's files

In [ ]:
#| export
def file_schema(kg:KnowledgeGraph): #Knowledge graph to query from
  """
  Display all properties for a file
  """
  query = """
  PREFIX spdx:<http://spdx.org/rdf/terms#>
  SELECT DISTINCT ?property
  WHERE {
    ?file rdf:type spdx:File .
    ?file ?property ?value .
  }
  order by asc(UCASE(str(?property)))
  """
  return kg.query_as_df(query)

In [ ]:
#| hide
file_schema(kg)

,property
0,spdx:annotation
1,spdx:checksum
2,spdx:copyrightText
3,spdx:fileContributor
4,spdx:fileName
5,spdx:fileType
6,spdx:licenseComments
7,spdx:licenseConcluded
8,spdx:licenseInfoInFile
9,spdx:noticeText


In [ ]:
#| export
from pandas import DataFrame
def get_files_data(kg:KnowledgeGraph #Knowledge graph to query from
                   ) -> DataFrame: #Return result to dataframe
    """
    Return all files and their properties
    """
    query = """
    PREFIX spdx:<http://spdx.org/rdf/terms#>
    SELECT 
    (?file AS ?fileID)
    ?fileName
    ?fileType
    ?licenseInFile
    (GROUP_CONCAT(?contributor; SEPARATOR=", ") AS ?contributors)
    (GROUP_CONCAT(?licenseConcluded; SEPARATOR=", ") AS ?licenseConcluded)
    ?checksum
    ?relationship
    ?annotation
    ?comment
    ?licenseComments
    ?noticeText
    WHERE {
    ?file rdf:type spdx:File .
    ?file spdx:fileName ?fileName .
    ?file spdx:fileContributor ?contributor .
    ?file spdx:licenseInfoInFile ?licenseInFile .
    ?file spdx:licenseConcluded ?licenseConcluded .
    ?file spdx:checksum ?checksum .
    ?file spdx:copyrightText ?copyrightText .
    ?file spdx:fileType ?fileType .
    OPTIONAL {?file spdx:relationship ?relationship .}
    OPTIONAL {?file spdx:annotation ?annotation . }
    OPTIONAL {?file rdfs:comment ?comment . }
    OPTIONAL {?file spdx:licenseComments ?licenseComments . }
    OPTIONAL {?file spdx:noticeText ?noticeText . }
    }
    GROUP BY ?file
    """

    return kg.query_as_df(query)

In [ ]:
#| hide
get_files_data(kg)

,fileID,fileName,fileType,licenseInFile,contributors,licenseConcluded,checksum,relationship,comment,licenseComments,noticeText,annotation
0,<http://spdx.org/spdxdocs/spdx-example-444504E...,./lib-source/jena-2.6.3-sources.jar,spdx:fileType_archive,<http://spdx.org/spdxdocs/spdx-example-444504E...,"Apache Software Foundation, Hewlett Packard Inc.",http://spdx.org/spdxdocs/spdx-example-444504E0...,_:N6461fac3564e43e1b17fda576e38059b,_:N31b5188e1bcf4a84b35bc57593575ea2,This file belongs to Jena,This license is used by Jena,NaN,NaN
1,<http://spdx.org/spdxdocs/spdx-example-444504E...,./lib-source/commons-lang3-3.1-sources.jar,spdx:fileType_archive,<http://spdx.org/licenses/Apache-2.0>,Apache Software Foundation,http://spdx.org/licenses/Apache-2.0,_:Nef29e6762d464fdbb2cd80d037b29bac,_:N8e33a181d6b94af09ec5d5fa352968cd,This file is used by Jena,NaN,Apache Commons Lang\nCopyright 2001-2011 The A...,NaN
2,<http://spdx.org/spdxdocs/spdx-example-444504E...,./src/org/spdx/parser/DOAPProject.java,spdx:fileType_source,<http://spdx.org/licenses/Apache-2.0>,"Source Auditor Inc., Black Duck Software In.c,...","http://spdx.org/licenses/Apache-2.0, http://sp...",_:N25fcc3b7ec55465a9a3b9b4d4b70c093,NaN,NaN,NaN,NaN,NaN
3,<http://spdx.org/spdxdocs/spdx-example-444504E...,./package/foo.c,spdx:fileType_source,<http://spdx.org/spdxdocs/spdx-example-444504E...,"IBM Corporation, IBM Corporation, Modified by ...","N65e487122a1b4d1fb914b8efbd909ad8, N65e487122a...",_:N1e8634a087544e39997f941eba52cc4d,_:Nd8abface2be04dd0a1f43b0327756089,The concluded license was taken from the packa...,The concluded license was taken from the packa...,Copyright (c) 2001 Aaron Lehmann aaroni@vitelu...,_:Nd3a64804312245779939924d8cbd9ada


In [ ]:
#| export
def get_files_graph(kg:KnowledgeGraph #Knowledge graph to query from
                    )-> KnowledgeGraph: #Return result to knowledge graph
    """
    Return a subgraph of the files and their properties
    """
    namespaces = {
        "spdx": "http://spdx.org/rdf/terms#"
    }
    subgraph = kglab.KnowledgeGraph(
        namespaces = namespaces,
    )
    query = """
    PREFIX spdx:<http://spdx.org/rdf/terms#>
    CONSTRUCT {
        ?file rdf:type spdx:File .
        ?file spdx:fileName ?fileName .
        ?file spdx:fileContributor ?contributor .
        ?file spdx:licenseInfoInFile ?licenseInFile .
        ?file spdx:checksum ?checksum .
        ?file spdx:relationship ?relationship .
    }
    WHERE {
        ?file rdf:type spdx:File .
        ?file spdx:fileName ?fileName .
        ?file spdx:fileContributor ?contributor .
        ?file spdx:licenseInfoInFile ?licenseInFile .
        OPTIONAL {?file spdx:relationship ?relationship .}
    }
    """

    for row in kg.query(query):
        s, p, o = row
        subgraph.add(s, p, o)

    return subgraph # Return the subgraph

## Packages



In [ ]:
#| export
def package_schema(kg:KnowledgeGraph #Knowledge graph to query from
                  )-> DataFrame: #Return result to dataframe
  """
  Return the schema for the package
  """
  query = """
  PREFIX spdx:<http://spdx.org/rdf/terms#>
  SELECT DISTINCT ?property
  WHERE {
    ?package rdf:type spdx:Package .
    ?package ?property ?value .
  }
  order by asc(UCASE(str(?property)))
  """
  return kg.query_as_df(query)

In [ ]:
#| hide
package_schema(kg)

,property
0,spdx:annotation
1,spdx:attributionText
2,spdx:checksum
3,spdx:copyrightText
4,spdx:description
5,spdx:downloadLocation
6,spdx:externalRef
7,spdx:filesAnalyzed
8,spdx:licenseComments
9,spdx:licenseConcluded


In [ ]:
#| export
def get_package_data(kg:KnowledgeGraph #Knowledge graph to query from
                     ) -> DataFrame: #Return result to dataframe
    """
    Construct a dataframe of package data
    """
    query = """
    PREFIX spdx:<http://spdx.org/rdf/terms#>
    SELECT ?package
        (GROUP_CONCAT(?annotation; SEPARATOR=", ") AS ?annotations)
        (GROUP_CONCAT(?attributionText; SEPARATOR=", ") AS ?attributionTexts)
        ?builtDate
        (GROUP_CONCAT(?checksum; SEPARATOR=", ") AS ?checksums)
        ?comment
        ?copyrightText
        ?description
        ?downloadLocation
        (GROUP_CONCAT(?externalRef; SEPARATOR=", ") AS ?externalRefs)
        ?filesAnalyzed
        (GROUP_CONCAT(?hasFile; SEPARATOR=", ") AS ?hasFiles)
        ?homepage
        ?licenseComments
        ?licenseConcluded
        ?licenseDeclared
        (GROUP_CONCAT(?licenseInfoFromFile; SEPARATOR=", ") AS ?licenseInfoFromFiles)
        ?name
        ?originator
        ?packageFileName
        ?packageVerificationCode
        ?primaryPackagePurpose
        ?releaseDate
        ?sourceInfo
        ?summary
        ?supplier
        ?validUntilDate
        ?versionInfo
        (GROUP_CONCAT(?relationship; SEPARATOR=", ") AS ?relationships)
    WHERE {
        ?package rdf:type spdx:Package .
        OPTIONAL { ?package spdx:annotation ?annotation . }
        OPTIONAL { ?package spdx:attributionText ?attributionText . }
        OPTIONAL { ?package spdx:builtDate ?buildDate . }
        OPTIONAL { ?package spdx:checksum ?checksum . }
        OPTIONAL { ?package spdx:comment ?comment . }
        OPTIONAL { ?package spdx:copyrightText ?copyrightText . }
        OPTIONAL { ?package spdx:description ?description . }
        OPTIONAL { ?package spdx:downloadLocation ?downloadLocation . }
        OPTIONAL { ?package spdx:externalRef ?externalRef . }
        OPTIONAL { ?package spdx:filesAnalyed ?filesAnalyzed . }
        OPTIONAL { ?package spdx:hasFile ?hasFile . }
        OPTIONAL { ?package spdx:homepage ?homepage . }
        OPTIONAL { ?package spdx:licenseComments ?licenseComments . }
        OPTIONAL { ?package spdx:licenseConcluded ?licenseConcluded . }
        OPTIONAL { ?package spdx:licenseDeclared ?licenseDeclared . }
        OPTIONAL { ?package spdx:licenseInfoFromFile ?licenseInfoFromFile . }
        OPTIONAL { ?package spdx:name ?name . }
        OPTIONAL { ?package spdx:originator ?originator . }
        OPTIONAL { ?package spdx:packageFileName ?packageFileName . }
        OPTIONAL { ?package spdx:packageVerificationCode ?packageVerificationCode . }
        OPTIONAL { ?package spdx:primaryPackagePurpose ?primaryPackagePurpose . }
        OPTIONAL { ?package spdx:releaseDate ?releaseDate . }
        OPTIONAL { ?package spdx:sourceInfo ?sourceInfo . }
        OPTIONAL { ?package spdx:summary ?summary . }
        OPTIONAL { ?package spdx:supplier ?supplier . }
        OPTIONAL { ?package spdx:validUntilDate ?validUntilDate . }
        OPTIONAL { ?package spdx:versionInfo ?versionInfo . }
        OPTIONAL { ?package spdx:relationship ?relationship . }
    }
    GROUP BY ?package
    """

    return kg.query_as_df(query)

In [ ]:
#| hide
get_package_data(kg)

,package,annotations,attributionTexts,checksums,copyrightText,downloadLocation,externalRefs,hasFiles,licenseConcluded,licenseDeclared,...,relationships,description,licenseComments,originator,packageFileName,packageVerificationCode,sourceInfo,summary,supplier,versionInfo
0,<http://spdx.org/spdxdocs/spdx-example-444504E...,,,,NOASSERTION,spdx:noassertion,,,spdx:noassertion,spdx:noassertion,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,<http://spdx.org/spdxdocs/spdx-example-444504E...,"Nbb6b66d4652d4ef09235002f98800b3b, Nbb6b66d465...",The GNU C Library is free software. See the f...,"N4e489edb005e44abacf86caa550a69b1, N4e489edb00...",Copyright 2008-2010 John Smith,http://ftp.gnu.org/gnu/glibc/glibc-ports-2.15....,"N8388716cbea149489f86f334e005e3b8, N8388716cbe...",,_:N3807ba3cfb01414faf73edfd4df99bb7,_:N7d77fa4c353443e2bfd15eb2997cf05d,...,"Na92eacc24d3f4a0bb3a4e74b6127b215, N6d8fe8fe70...",The GNU C Library defines functions that are s...,The license for this project changed with the ...,Organization: ExampleCodeInspect (contact@exam...,glibc-2.11.1.tar.gz,_:N94f3dba040984a8abea198c8804e3d30,uses glibc-2_11-branch from git://sourceware.o...,GNU C library.,Person: Jane Doe (jane.doe@example.com),2.11.1
2,<http://spdx.org/spdxdocs/spdx-example-444504E...,,,Nb947798189854e39b823025c46c169ea,Copyright Saxonica Ltd,https://sourceforge.net/projects/saxon/files/S...,,,<http://spdx.org/licenses/MPL-1.0>,<http://spdx.org/licenses/MPL-1.0>,...,,The Saxon package is a collection of tools for...,Other versions available for a commercial license,NaN,saxonB-8.8.zip,NaN,NaN,NaN,NaN,8.8
3,<http://spdx.org/spdxdocs/spdx-example-444504E...,,,,NOASSERTION,https://search.maven.org/remotecontent?filepat...,Nea326bcfe19f4790ab4353e1b52aef52,,spdx:noassertion,spdx:noassertion,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.12.0


In [ ]:
#| export
def get_package_graph(kg:KnowledgeGraph #Knowledge graph to query from
                      ) -> KnowledgeGraph: #Return result to knowledge graph
    """
    Construct a subgraph of the package data
    """
    namespaces = {
        "spdx": "http://spdx.org/rdf/terms#"
    }   

    subgraph = kglab.KnowledgeGraph(
        namespaces = namespaces,
    )   

    query = """
    PREFIX spdx:<http://spdx.org/rdf/terms#>
    CONSTRUCT {
        ?package rdf:type spdx:Package .
        ?package spdx:annotation ?annotation .
        ?package spdx:attributionText ?attributionText .
        ?package spdx:builtDate ?buildDate .
        ?package spdx:checksum ?checksum .
        ?package spdx:comment ?comment .
        ?package spdx:copyrightText ?copyrightText .
        ?package spdx:description ?description .
        ?package spdx:downloadLocation ?downloadLocation .
        ?package spdx:externalRef ?externalRef .
        ?package spdx:filesAnalyed ?filesAnalyzed .
        ?package spdx:hasFile ?hasFile .
        ?package spdx:homepage ?homepage .
        ?package spdx:licenseComments ?licenseComments .
        ?package spdx:licenseConcluded ?licenseConcluded .
        ?package spdx:licenseDeclared ?licenseDeclared .
        ?package spdx:licenseInfoFromFile ?licenseInfoFromFile .
        ?package spdx:name ?name .
        ?package spdx:originator ?originator .
        ?package spdx:packageFileName ?packageFileName .
        ?package spdx:packageVerificationCode ?packageVerificationCode .
        ?package spdx:primaryPackagePurpose ?primaryPackagePurpose .
        ?package spdx:releaseDate ?releaseDate .
        ?package spdx:sourceInfo ?sourceInfo .
        ?package spdx:summary ?summary .
        ?package spdx:supplier ?supplier .
        ?package spdx:validUntilDate ?validUntilDate .
        ?package spdx:versionInfo ?versionInfo .
        ?package spdx:relationship ?relationship .
    }   
    WHERE {
        ?package rdf:type spdx:Package .
        OPTIONAL { ?package spdx:annotation ?annotation . }
        OPTIONAL { ?package spdx:attributionText ?attributionText . }
        OPTIONAL { ?package spdx:attributionText ?attributionText . }
        OPTIONAL { ?package spdx:builtDate ?buildDate . }
        OPTIONAL { ?package spdx:checksum ?checksum . }
        OPTIONAL { ?package spdx:comment ?comment . }
        OPTIONAL { ?package spdx:copyrightText ?copyrightText . }
        OPTIONAL { ?package spdx:description ?description . }
        OPTIONAL { ?package spdx:downloadLocation ?downloadLocation . }
        OPTIONAL { ?package spdx:externalRef ?externalRef . }
        OPTIONAL { ?package spdx:filesAnalyed ?filesAnalyzed . }
        OPTIONAL { ?package spdx:hasFile ?hasFile . }
        OPTIONAL { ?package spdx:homepage ?homepage . }
        OPTIONAL { ?package spdx:licenseComments ?licenseComments . }
        OPTIONAL { ?package spdx:licenseConcluded ?licenseConcluded . }
        OPTIONAL { ?package spdx:licenseDeclared ?licenseDeclared . }
        OPTIONAL { ?package spdx:licenseInfoFromFile ?licenseInfoFromFile . }
        OPTIONAL { ?package spdx:name ?name . }
        OPTIONAL { ?package spdx:originator ?originator . }
        OPTIONAL { ?package spdx:packageFileName ?packageFileName . }
        OPTIONAL { ?package spdx:packageVerificationCode ?packageVerificationCode . }
        OPTIONAL { ?package spdx:primaryPackagePurpose ?primaryPackagePurpose . }
        OPTIONAL { ?package spdx:releaseDate ?releaseDate . }
        OPTIONAL { ?package spdx:sourceInfo ?sourceInfo . }
        OPTIONAL { ?package spdx:summary ?summary . }
        OPTIONAL { ?package spdx:supplier ?supplier . }
        OPTIONAL { ?package spdx:validUntilDate ?validUntilDate . }
        OPTIONAL { ?package spdx:versionInfo ?versionInfo . }
        OPTIONAL { ?package spdx:relationship ?relationship . }
    }
    GROUP BY ?package
    """
    for row in kg.query(query):
        s, p, o = row
        subgraph.add(s, p, o)

    return subgraph # Return the subgraph

## Relationships

In [ ]:
#| export
def relationship_schema(kg : KnowledgeGraph #Knowledge graph to query from
                        ) -> DataFrame: #Return result to dataframe
    """
    Return the schema for the relationships
    """

    query = """
    PREFIX spdx:<http://spdx.org/rdf/terms#>
    SELECT DISTINCT ?property
    WHERE {
    ?file rdf:type spdx:Relationship .
    ?file ?property ?value .
    }
    """


    return kg.query_as_df(query)


In [ ]:
#| hide
relationship_schema(kg)

,property
0,rdf:type
1,spdx:relationshipType
2,spdx:relatedSpdxElement


In [ ]:
#| export
def get_relationship_data(kg:KnowledgeGraph #Knowledge graph to query from
                          ) -> DataFrame: #Return result to dataframe
    
    query = """
    PREFIX spdx:<http://spdx.org/rdf/terms#>
    SELECT ?element ?elementType ?relationshipType ?relatedElement ?relatedElementType
    WHERE {
    ?element spdx:relationship ?relationship .
    ?element rdf:type ?elementType .
    ?relationship spdx:relatedSpdxElement ?relatedElement .
    ?relationship spdx:relationshipType ?relationshipType .
    ?relatedElement rdf:type ?relatedElementType .
    }
    """

    return kg.query_as_df(query)

In [ ]:
#| hide
get_relationship_data(kg)

,element,elementType,relationshipType,relatedElement,relatedElementType
0,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:File,spdx:relationshipType_contains,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package
1,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package,spdx:relationshipType_contains,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:File
2,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package,spdx:relationshipType_dynamicLink,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package
3,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package,spdx:relationshipType_contains,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:File
4,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package,spdx:relationshipType_contains,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:File
5,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:SpdxDocument,spdx:relationshipType_contains,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package
6,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:File,spdx:relationshipType_generatedFrom,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package
7,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:SpdxDocument,spdx:relationshipType_describes,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:File
8,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:SpdxDocument,spdx:relationshipType_describes,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package


In [ ]:
#| export
def get_relationship_graph(kg: KnowledgeGraph #Knowledge graph to query from
                           ) -> KnowledgeGraph: #Return result to knowledge graph
    """
    Construct a subgraph of the relationship data
    """

    namespaces = {
    "spdx": "http://spdx.org/rdf/terms#",
    "this": "http://spdx.org/spdxdocs/spdx-example-444504E0-4F89-41D3-9A0C-0305E82C3301#"
    }
    subgraph = kglab.KnowledgeGraph(
        namespaces = namespaces,
    )
    query = """
    PREFIX spdx:<http://spdx.org/rdf/terms#>
    CONSTRUCT {
    ?element spdx:relationship ?relationship .
    ?element rdf:type ?elementType .
    ?relationship spdx:relatedSpdxElement ?relatedElement .
    ?relationship spdx:relationshipType ?relationshipType .
    ?relatedElement rdf:type ?relatedElementType .
    }
    WHERE {
    ?element spdx:relationship ?relationship .
    ?element rdf:type ?elementType .
    ?relationship spdx:relatedSpdxElement ?relatedElement .
    ?relationship spdx:relationshipType ?relationshipType .
    ?relatedElement rdf:type ?relatedElementType .
    }
    """

    for row in kg.query(query):
        s, p, o = row
        subgraph.add(s, p, o)

    return subgraph

In [ ]:
#| hide
get_relationship_graph(kg)

<kglab.kglab.KnowledgeGraph>

## Visualization

Functions to visualize graph results

In [ ]:
#| export
def visualize_graph(kg:KnowledgeGraph, #Knowledge graph to query from
                    filename: str="tmp.html", #Optional, filename to save graph to
                    VIS_STYLE: str={} #Optional, visualization style
                    ):
    """
    Visualize the knowledge graph
    """
    if not VIS_STYLE:
        VIS_STYLE = {
            "spdx": {
                "color": "orange",
                "size": 40,
            },
            "rdf":{
                "color": "blue",
                "size": 30,
            },
            "ptr":{
                "color": "red",
                "size": 20,
            },
        }
    subgraph = kglab.SubgraphTensor(kg)
    pyvis_graph = subgraph.build_pyvis_graph(notebook=True, style=VIS_STYLE)
    pyvis_graph.force_atlas_2based()
    pyvis_graph.show(filename)

In [ ]:
#| hide
visualize_graph(kg)

tmp.html


In [ ]:
#| default_exp core